In [1]:
"""
Format Dataset: (list of lists) 
    [
        [context_0, context_1, ..., context_N, response], # sample_0
        [context_0, context_1, ..., context_N, response], # sample_1
        ...,
        [context_0, context_1, ..., context_N, response] # sample_L
    ]
"""

'\nFormat Dataset: (list of lists) \n    [\n        [context_0, context_1, ..., context_N, response], # sample_0\n        [context_0, context_1, ..., context_N, response], # sample_1\n        ...,\n        [context_0, context_1, ..., context_N, response] # sample_L\n    ]\n'

In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split

# tokenizer
from transformers import (
    MODEL_WITH_LM_HEAD_MAPPING,
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    AutoModelWithLMHead,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
    get_linear_schedule_with_warmup,
)

In [3]:
PATH_DATASET = "0.0. prepare_dataset.csv"

# Model
DialogGPT_MODEL = "microsoft/DialoGPT-small" 

In [4]:
# Instance Model & Tokenizer
tokenizer = AutoTokenizer.from_pretrained(DialogGPT_MODEL)

In [6]:
df = pd.read_csv(PATH_DATASET)
df.head(5)

,context_0,context_1,context_2,context_3,context_4,context_5,context_6,response
0,Morty! You gotta come on. Jus'... you gotta co...,"What, Rick? What’s going on?","I got a surprise for you, Morty.",It's the middle of the night. What are you tal...,"Come on, I got a surprise for you. Come on, h...",Ow! Ow! You're tugging me too hard!,"We gotta go, gotta get outta here, come on. Go...","What do you think of this... flying vehicle, M..."
1,"What, Rick? What’s going on?","I got a surprise for you, Morty.",It's the middle of the night. What are you tal...,"Come on, I got a surprise for you. Come on, h...",Ow! Ow! You're tugging me too hard!,"We gotta go, gotta get outta here, come on. Go...","What do you think of this... flying vehicle, M...","Yeah, Rick... I-it's great. Is this the surprise?"
2,"I got a surprise for you, Morty.",It's the middle of the night. What are you tal...,"Come on, I got a surprise for you. Come on, h...",Ow! Ow! You're tugging me too hard!,"We gotta go, gotta get outta here, come on. Go...","What do you think of this... flying vehicle, M...","Yeah, Rick... I-it's great. Is this the surprise?",Morty. I had to... I had to do it. I had— I ha...
3,It's the middle of the night. What are you tal...,"Come on, I got a surprise for you. Come on, h...",Ow! Ow! You're tugging me too hard!,"We gotta go, gotta get outta here, come on. Go...","What do you think of this... flying vehicle, M...","Yeah, Rick... I-it's great. Is this the surprise?",Morty. I had to... I had to do it. I had— I ha...,What?! A bomb?!
4,"Come on, I got a surprise for you. Come on, h...",Ow! Ow! You're tugging me too hard!,"We gotta go, gotta get outta here, come on. Go...","What do you think of this... flying vehicle, M...","Yeah, Rick... I-it's great. Is this the surprise?",Morty. I had to... I had to do it. I had— I ha...,What?! A bomb?!,We're gonna drop it down there just get a whol...


In [7]:
TEST_SIZE = 0.1

Split Dataset in train & test

In [8]:
df_train, df_val = train_test_split(df,test_size=TEST_SIZE, random_state=0)

In [13]:
# transform dataset
def construct_conv(row, tokenizer, eos = True):
    """ Add EOS token to each sentecenes from row --> tokenized ---> flatten tokenized row
    Args:
        row (list of sentences): the first N elements is the context and the last element is the response or target
        tokenizer (object): tokenizer from respective model
        eos (bool, optional): end of sentence. Defaults to True.

    Returns:
        (list of ints): [len(flatten(row))] (shape is variable because not have padding) flatten row tokenized 
            >>>>  [651,5513,86,24905,287,...,640,284,651,5513,86,24905] --> 2495=eos_token
    """
    flatten = lambda l: [item for sublist in l for item in sublist]
    conv = [tokenizer.encode(x) + [tokenizer.eos_token_id] for x in row]
    conv = flatten(conv) 
    return conv


import os
from torch.utils.data import Dataset
import logging
logger = logging.getLogger(__name__)

class ConversationDataset(Dataset):
    def __init__(self, tokenizer, args, df, block_size=512):

        block_size = block_size - (tokenizer.max_len - tokenizer.max_len_single_sentence)

        directory = "cached"
        cached_features_file = os.path.join(
            directory, args.model_type + "_cached_lm_" + str(block_size)
        )

        if os.path.exists(cached_features_file) and not args.overwrite_cache:
            logger.info("Loading features from cached file %s", cached_features_file)
            with open(cached_features_file, "rb") as handle:
                self.examples = pickle.load(handle)
        else:
            logger.info("Creating features from dataset file at %s", directory)

            self.examples = []
            for _, row in df.iterrows():
                conv = construct_conv(row, tokenizer)
                self.examples.append(conv)

            logger.info("Saving features into cached file %s", cached_features_file)
            with open(cached_features_file, "wb") as handle:
                pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return torch.tensor(self.examples[item], dtype=torch.long)

In [16]:
model_type = 'gpt2'
block_size=512

block_size = block_size - (tokenizer.max_len - tokenizer.max_len_single_sentence)

directory = "cached"
cached_features_file = os.path.join(
    directory, model_type + "_cached_lm_" + str(block_size)
    )

AttributeError: 'GPT2TokenizerFast' object has no attribute 'max_len'

In [17]:
tokenizer.max_len

AttributeError: 'GPT2TokenizerFast' object has no attribute 'max_len'